# 宽基指数和行业分布

## 导入模块

In [11]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plts
import feather
import sunlandsdatasdk as sd

## 读入数据

In [56]:
date = '20190102'
jump = feather.read_dataframe(f'../data/jump/2019/jump_{date}.feather')
jump['date'] = pd.to_datetime(jump['date'])
jump = jump[jump['jump']]

In [57]:
date_series = jump['date'].sort_values().unique()

sd.auth('*', '*')
hs800 = pd.DataFrame(columns=['issue', 'date'])
hs1000 = pd.DataFrame(columns=['issue', 'date'])
for date in date_series:
    hs800_daily = pd.DataFrame(columns=['issue', 'date'])
    hs800_daily['issue'] = sd.get_index_stocks('000906', weight=False, date=date)
    hs800_daily['date'] = date
    if hs800.empty:
        hs800 = hs800_daily.copy()
    else:
        hs800 = pd.concat([hs800, hs800_daily])
    hs1000_daily = pd.DataFrame(columns=['issue', 'date'])
    hs1000_daily['issue'] = sd.get_index_stocks('000852', weight=False, date=date)
    hs1000_daily['date'] = date
    if hs1000.empty:
        hs1000 = hs1000_daily
    else:
        hs1000 = pd.concat([hs1000, hs1000_daily])

## 宽基指数

In [59]:
jump_hs800 = pd.merge(
    jump,
    hs800,
    on=['issue', 'date'],
    how='inner'
)

jump_hs1000 = pd.merge(
    jump,
    hs1000,
    on=['issue', 'date'],
    how='inner'
)

### 跳跃数量统计

### 跳跃幅度统计

## 行业分布